In [427]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [375]:
baseUrl = 'https://www.nfl.com'
teamAbbr = pd.Series(["SF","CHI","CIN","BUF","DEN","CLE","TB","ARI","LAC","KC","IND","DAL","MIA","PHI","ATL","NYG",
                     "JAX","NYJ","DET","GB","CAR","NE","OAK","LAR","BAL","WAS","NO","SEA","PIT","HOU","TEN","MIN"])

In [499]:
playerPayload = {"type":"/stats","subject":"player-stats","category":"category",
                 "statisticCategory":"rushing","season":"2019",
                 "seasonType":"REG",
                 "list":"all",
                 "specificCategory":"rushingYards",
                 "order":"DESC"}

teamPayload = {"type":"/stats","subject":"team-stats","category":"offense",
               "statisticCategory":"passing",
               "season":"2019",
               "seasonType":"REG",
               "list":"all",
               "specificCategory":"",
               "order":""}

schedulePayload = {"type":"/schedules",
                  "season":"2019",
                  "seasonType":"REG"}


In [500]:
payload = schedulePayload

url = 'https://www.nfl.com'
for key in payload:
    if payload[key]:
        url += payload[key]
        url += '/'
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')

200 https://www.nfl.com/schedules/2019/REG/


In [501]:
season = soup.find_all('select', {"class":"d3-o-dropdown"})[1].find_all("option")
seasonWeeks = []
for week in season:
    seasonWeeks.append(week.get("value"))
current = soup.find_all('select', {"class":"d3-o-dropdown"})[1].find('option', {"selected":True}).get("value")
print(current)
print(current.rsplit('/', 2)[-2])

/schedules/2019/REG1/
REG1


In [324]:
tableRows = []
pagingText = True


while pagingText:
    time.sleep(2)
    response = requests.get(url)
    print("url:",response.url)
    soup = BeautifulSoup(response.text,'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')

    for row in rows:
        cols = row.find_all('td')
        if cols:
            cols = [ele.text.strip() for ele in cols]
            tableRows.append([ele for ele in cols if ele]) # Get rid of empty values
    pagingText = soup.find("a", {"class": "nfl-o-table-pagination__next"})
    if pagingText:
        url = baseUrl + pagingText.get('href')

url: https://www.nfl.com/stats/team-stats/offense/passing/2019/reg/all


In [330]:
resultsDf = pd.DataFrame(tableRows, columns=columnHeader)
# remove \n whitespace from team names
find = re.compile(r"^(\n*).*")
resultsDf['Team'] = resultsDf['Team'].apply(lambda x: re.search(find,x).group())
pd.DataFrame.head(resultsDf)

,Team,Att,Cmp,Cmp %,Yds/Att,Pass Yds,TD,INT,Rate,1st,1st%,20+,40+,Lng,Sck,SckY
0,Redskins,479,298,62.2,6.7,3205,18,13,83,154,32.2,38,6,75T,50,393
1,Buccaneers,630,382,60.6,8.1,5127,33,30,84.1,244,38.7,75,13,71T,47,282
2,Seahawks,517,341,66,7.9,4110,31,6,105.3,190,36.8,55,10,60T,48,319
3,49ers,478,331,69.2,8.4,4029,28,13,103.1,195,40.8,62,8,75T,36,237
4,Chargers,597,394,66,7.8,4648,24,20,89,220,36.9,57,12,84T,34,222


In [544]:
def getStats(url, payload): 
    tableRows = []
    pagingText = True
    
    for key in payload:
        if payload[key]:
            url += payload[key]
            url += '/'
    response = requests.get(url)
    
    # get column names
    soup = BeautifulSoup(response.text,'html.parser')
    columnHeader = []
    header = soup.find('table').find_all('th')
    for col in header:
        columnHeader.append(col.text)
    columnHeader = [c.strip('\n') for c in columnHeader]

    # search through pages and append to tableRows
    while pagingText:
        time.sleep(2)
        response = requests.get(url)
        print("url:",response.url)
        soup = BeautifulSoup(response.text,'html.parser')
        table = soup.find('table')
        rows = table.find_all('tr')

        for row in rows:
            cols = row.find_all('td')
            if cols:
                cols = [ele.text.strip() for ele in cols]
                cols = [ele.rstrip('\n') for ele in cols]
                tableRows.append([ele for ele in cols if ele]) # Get rid of empty values
        pagingText = soup.find("a", {"class": "nfl-o-table-pagination__next"})
        if pagingText:
            url = baseUrl + pagingText.get('href')
    
    resultsDf = pd.DataFrame(tableRows, columns=columnHeader)
    find = re.compile(r"^(\n*).*")
    resultsDf['Team'] = resultsDf['Team'].apply(lambda x: re.search(find,x).group())
    
    # add teamId to table
    resultsDf = resultsDf.sort_values(by=['Team']).reset_index(drop=True)
    resultsDf['teamId'] = teamAbbr
    return resultsDf

In [540]:
def getMatchups(url,payload):
    for key in payload:
        if payload[key]:
            url += payload[key]
            url += '/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    
    matchups = []
    columnHeader = ['matchupId', 'team1', 'team2', 'team1Score','team2Score','weekType','weekNum']
    season = soup.find_all('select', {"class":"d3-o-dropdown"})[1].find_all("option")
    seasonWeeks = []
    for week in season:
        seasonWeeks.append(week.get("value"))
    for week in seasonWeek:
        url = baseUrl + week
        weekName = week.rsplit('/', 2)[-2]
        weekType = re.findall("[a-zA-Z]+", weekName)[0]
        weekNum = re.findall(r'\d+', weekName)[0]
        games = soup.find_all('a', {"class": "nfl-c-matchup-strip__game"})
        index = 0
        for game in games:
            gameSoup = game.find_all('span', {"class":"nfl-c-matchup-strip__team-abbreviation"})
            matchup = []
            matchup.append(index)
            for team in gameSoup:
                matchup.append(re.sub(r'[^A-Za-z]', '', team.get_text()))
            scoreSoup = game.find_all('div', {'class':"nfl-c-matchup-strip__team-score"})
            if scoreSoup:
                for team in scoreSoup:
                    matchup.append(team.get_text())
            else:
                matchup.append("")
                matchup.append("")
            matchup.append(weekType)
            matchup.append(weekNum)
            matchups.append(matchup)
            index += 1
    matchupsDf = pd.DataFrame(matchups,columns=columnHeader)
    return matchupsDf
#matchupsDf.loc[(matchupsDf['weekType']=='REG') & (matchupsDf['weekNum']=='1')]


In [547]:
teamPayload = {"stats":"/stats","subject":"team-stats",
               "category":"offense",
               "statisticCategory":"passing",
               "season":"2019",
               "seasonType":"REG",
               "list":"all",
               "specificCategory":"",
               "order":""}
df1 = getStats(baseUrl,teamPayload)
pd.DataFrame.head(df1)

url: https://www.nfl.com/stats/team-stats/offense/passing/2019/reg/all


,Team,Att,Cmp,Cmp %,Yds/Att,Pass Yds,TD,INT,Rate,1st,1st%,20+,40+,Lng,Sck,SckY,teamId
0,49ers,478,331,69.2,8.4,4029,28,13,103.1,195,40.8,62,8,75T,36,237,SF
1,Bears,580,371,64,6.2,3573,20,12,83.9,178,30.7,39,2,53,45,282,CHI
2,Bengals,616,356,57.8,6.5,3994,18,16,76.2,191,31,48,7,66T,48,342,CIN
3,Bills,513,299,58.3,6.8,3476,21,12,82.8,162,31.6,54,8,53T,40,247,BUF
4,Broncos,504,312,61.9,6.7,3401,16,10,84.1,162,32.1,46,10,75T,41,286,DEN


In [550]:
schedulePayload = {"type":"/schedules",
                  "season":"2019",
                  "seasonType":"REG"}
df2 = getMatchups(baseUrl,schedulePayload)
df2.loc[(df2['weekType']=='REG') & (df2['weekNum']=='1')]

,matchupId,team1,team2,team1Score,team2Score,weekType,weekNum
80,0,GB,CHI,10,3,REG,1
81,1,WAS,PHI,27,32,REG,1
82,2,BUF,NYJ,17,16,REG,1
83,3,ATL,MIN,12,28,REG,1
84,4,BAL,MIA,59,10,REG,1
85,5,KC,JAX,40,26,REG,1
86,6,TEN,CLE,43,13,REG,1
87,7,LA,CAR,30,27,REG,1
88,8,CIN,SEA,20,21,REG,1
89,9,IND,LAC,24,30,REG,1
